# Favouritism

It could be useful to identify unusual patterns of behaviour in various actors, such as one doctor using a lab for tests that is not frequently used by others. While such behaviour may be innocent, it is possible that two malicious sources may use each other as back up in order to disguise incorrect information. For example, a doctor may ask a lab technician to sign off on lab tests that haven't actually been done in order to clear a backlog of paperwork or further process a patient, even though this is against the hospital's rules.

A *favours* event occurs when a medical actor requests a test from a specific lab, and there is more than one lab available which is authorised to perform such a test. The event predicate specifies the request ID, requester, type of test, lab selected and labs overlooked. This allows statistics of favouritism to be generated, such as via the rateFavoured fluent, which records the number of times one lab was favoured out of the times that it was available. We also calculate the percentage of times a given lab was available out of all labs authorised to perform a given test.

In this notebook, we make use of a binomial test with a 5% significance level in order to assess whether the rate at which a lab is selected / favoured over other labs is unlikely given that we assume the rate of selection is related to the rate of availability.

This notebook contains predicates which are most relevant to a basic favouritism scenario. Many predicates, such as authorisation levels for doctors, nurses and other medical staff, have been left out to simplify this example.

### Set up:

In [ ]:
?- cd('~/work'), ['dec'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

### Labs + Actors

Request events initiate requested fluents. A requested fluent is multi valued, and is set to NA until a response to the request has been received. Requests and responses in the same message chain have the same reference value. Requests have a type, such as **history** for a patient history request, or **HIV test**. **Patient** refers to the patient that the request concerns, while **Source** is the laboratory or other hospital which is asked for information. The medical actor who requests the test is the **Requester**.

In [ ]:
% File: PatientCareProcedure.pl

% History requests are not yet fulfilled by default. History grants answer history requests, and result messages answer test requests.
initiates(request(Reference,Type,Patient,Source,Requester),requested(Reference,Type,Patient,Source,Requester)='NA',_).
initiates(response(Reference,_,Source), requested(Reference,Type,Patient,Source,Requester)='ANS',T):- 
holdsAt(requested(Reference,Type,Patient,Source,Requester)='NA',T).

We have three medical staff, **AW**, **EH** and **GG**, all of whom are initially available.

In [ ]:
% File: MedStaffNarrative.pl

% Some starting staff members.
initially(medStaff('AW')=nurse).
initially(medStaff('EH')=nurse).
initially(medStaff('GG')=specialist).
initially(available('AW')).
initially(available('EH')).
initially(available('GG')).


We also have three labs, **invercargill**, **wellington** and **otagoBlood**, all of whom are initially available. Each of these labs is certified by this hospital to perform HIV tests. (The details of test certifications will be covered in a separate notebook.) 

In [ ]:
% File: Labs.pl
initially(available("invercargill")).
initially(available("wellington")).
initially(available("otagoBlood")).

initially(certified("otagoBlood", 'HIV test',me)).
initially(certified("wellington", 'HIV test',me)).
initially(certified("invercargill", 'HIV test',me)).

initiates(certification(Lab,Test,Certifier), certified(Lab,Test,Certifier),_).
terminates(revokeCertification(Lab,Test,Certifier), certified(Lab,Test,Certifier),_).

Medical actors can clock on or off, changing whether or not they are available. Likewise, labratories can reconnect or withdraw their services from a hospital, affecting their availability.

In [ ]:
% File: Availability.pl
initiates(clockOn(Actor), available(Actor),_).
terminates(clockOff(Actor), available(Actor),_).
initiates(reconnects(Lab), available(Lab),_).
terminates(withdraws(Lab), available(Lab), _).

### Narrative

In our narrative, a history request is made at time 1 by **GG**. HIV test requests to **otagoBlood** and **wellington** by **AW** and **EH** follow at time period 2. At time period 2, **wellington** withdraws, meaning that it is not available for future test requests from time period 3 and onwards until the lab reconnects. At time period 3, **invercargill** also withdraws. At time period 3, while **invercargill** is still available, **EH** sends the lab an HIV test request, and at time period 4, **GG** sends a test request to **otagoBlood**.

At time period 4, **wellington** and **invercargill** reconnect and are available for test requests again. At time period 5, **GG** sends 4 **HIV** tests for different patients, which are all sent to **otagoBlood**. 

As patient history requests would be largely dictated by the patient's historic place of residence, we do not consider history requests in our favouritism calculations. For illustration purposes, all HIV test requests have been given a value of 1 here, although we would expect that a reference value would be unique to a particular message chain.

In [ ]:
% File: narrative.pl
happensAtNarrative(request(2,'history',123,"wellington", 'GG'),1).

happensAtNarrative(request(1, 'HIV test',123,"otagoBlood", 'AW'),2).
happensAtNarrative(request(1, 'HIV test',124,"otagoBlood", 'EH'),2).
happensAtNarrative(request(1, 'HIV test',126,"otagoBlood", 'EH'),2).
happensAtNarrative(request(1, 'HIV test',127,"wellington", 'EH'),2).
happensAtNarrative(withdraws("wellington"),2).

happensAtNarrative(withdraws("invercargill"),3).
happensAtNarrative(request(1, 'HIV test',127,"invercargill", 'EH'),3).

happensAtNarrative(request(1, 'HIV test',127,"otagoBlood", 'GG'),4).
happensAtNarrative(reconnects("wellington"),4).
happensAtNarrative(reconnects("invercargill"),4).

happensAtNarrative(request(1, 'HIV test',120,"otagoBlood", 'GG'),5).
happensAtNarrative(request(1, 'HIV test',121,"otagoBlood", 'GG'),5).
happensAtNarrative(request(1, 'HIV test',119,"otagoBlood", 'GG'),5).
happensAtNarrative(request(1, 'HIV test',118,"otagoBlood", 'GG'),5).

A *favours* event occurs when a particular medical staff member chooses a particular lab to request a medical test from, and there is at least one other lab also available at the same time period which is authorised to perform the same test.

### Favour predicates

In [ ]:
% File: favours.pl

happensAtNarrative(favours(Requester,Source, Others, Type, Reference),T):- happensAtNarrative(request(Reference,Type,Patient,Source,Requester),T), 
Type \= history, findall(Lab, (holdsAt(available(Lab),T), Lab \== Source, holdsAt(certified(Lab,Type,me),T)), Others), Others \= [].

The favoured predicate records the number of favours events that have occurred for each combination of requester, source, overlooked labs, type of medical test and reference number. This allows us to check the favoured predicate for a given medical staff member, a particular series of messages, or other criteria.

In [ ]:
% File: favoured.pl

initiates(favours(Requester,Source, Others, Type, Reference), favoured(Requester,Source, Others, Type, Reference)=NewVal, T):- 
(holdsAt(favoured(Requester,Source, Others, Type, Reference)=OV,T) -> OldVal = OV; OldVal = 0), aggregate_all(count, 
happensAtNarrative(favours(Requester,Source, Others, Type, Reference),T), Count), NewVal is OldVal + Count.

The rateFavoured fluent records the rate at which a given requester (medical staff member) has favoured a particular lab in a test request over at least one other available lab. Due to favoured predicates existing for the same favoured lab but with different lists of overlooked labs, we store our solutions using *findall* and sort the resulting list in order to remove duplicate solutions. 

For each case in which a given lab was favoured over other labs, we count up the number of times this lab was chosen over other labratories, and the number of times it was not picked.

In [ ]:
% File: rateFavoured.pl

holdsAt(rateFavoured(Requester, Source, Type, Reference,Success,Trials),T):- findall([S,Succ, Tr, Req, Ty, Ref], 
(holdsAt(favoured(Req,S, _, Ty, Ref)=_,T), setof(Pos, findall(ValN, (holdsAt(favoured(Req, S, _, Ty, Ref)=ValN,T)), Pos), P), P = [X], 
sumlist(X,Succ), setof(Neg, findall(ValM, (holdsAt(favoured(Req, Source2, List, Ty, Ref)=ValM,T), member(S, List), Source2 \== S), Neg), N), 
N = [Y], sumlist(Y,NSum), Tr is Succ + NSum), FA), sort(FA,Z), member(M,Z), M = [Source, Success, Trials, Requester, Type, Reference].

### Availability predicates

We record the number of times a given lab was available when a *favours* event takes place concerning our specified test. In the availabilityCount fluent, we include information on which lab was selected and which labs were available but not selected.

The availabilityPercent fluent records the percentage of total availability counts for a specific lab at out of all available labs' availability counts for the same medical actor, type of request and message reference.

In [ ]:
% File: availabilityCount.pl

initiates(favours(Requester,Source, _, Type, Reference), availabilityCount(Requester, Source, Type, Reference)=Count,T):- 
(holdsAt(availabilityCount(Requester, Source, Type, Reference)=OV,T) 
-> OldVal = OV; OldVal = 0), aggregate_all(count, happensAtNarrative(favours(Requester,Source, _, Type, Reference),T), Agg1), 
aggregate_all(count, 
(happensAtNarrative(favours(Requester,_, Others, Type, Reference),T), member(Source, Others)), Agg2), Count is OldVal + Agg1 + Agg2.

initiates(favours(Requester,_, Others, Type, Reference), availabilityCount(Requester, Source,  Type, Reference)=Count,T):- member(Source, Others), 
(holdsAt(availabilityCount(Requester, Source, Type, Reference)=OV,T) -> OldVal = OV; OldVal = 0), 
aggregate_all(count, happensAtNarrative(favours(Requester,Source, _, Type, Reference),T), Agg1), 
aggregate_all(count, (happensAtNarrative(favours(Requester,_, Others2, Type, Reference),T), member(Source, Others2)), Agg2), 
Count is OldVal + Agg1 + Agg2.

holdsAt(availabilityPercent(Requester, Source, Type, Reference,Percentage),T):- holdsAt(availabilityCount(Requester, Source,Type, Reference)=Count,T), 
findall(Other, (holdsAt(availabilityCount(Requester, Source2,Type, Reference)=Other,T), Source2 \= Source), OL), sumlist(OL,S), Count + S > 0, 
Percentage is Count / (Count + S).


### Binomial test

In order to determine whether a doctor is unfairly favouring a lab over other available labs, we perform a binomial test. This requires some helper predicates for factorial calculations, generating a list from 0 to N and splitting a list.

A binomial test involves calculating the cumulative probability of the actual number of successes, or a more extreme result under the given number of trials, and comparing this to a specified significance level (often 0.05). If the cumulative probability is greater than our siginficance level, we do not have evidence to reject the null hypothesis that the probability of success per trial is the same as specified. If the cumulative probability is less than the significance level, we have evidence to reject the null hypothesis.

In [ ]:
% File: binomial.pl


% Helper predicates:

n_factorial(0, 1).
n_factorial(N, F) :-
        N > 0,
        N1 is N - 1,
        n_factorial(N1, F1),
        F is N * F1.

% Create list from N to zero
pred(0, [0]).
pred(N, [N|T]) :-
    N > 0,
    N1 is N-1,
    pred(N1, T).

% Create list from zero to N
predRev(P,R):- pred(P, Res),reverse(R,Res).

% Split the list into left and right sublists, where Index refers to the starting position of the right sublist.
split(Index, List, Left, Right):- length(Left, Index), append(Left, Right, List).

% Probability of X Successes out of Tr Trials given a probability of success per trial of P:
p_binom(X,P,Tr,Ans):- P1 is 1 - P, n_factorial(Tr,FacN), n_factorial(X,FacX), Dif is Tr - X, n_factorial(Dif, FacDif), 
Ans is FacN*P^X*P1^Dif/(FacDif*FacX).

% Cumulative Probability of either less than or equal to / greater than or equal to X successes out of Tr trials.
% Specifying Dir = high will give the upper tail probability, while any other value for Dir will give the lower tail probability.
p_cumul_binom(X, P, Tr, Dir, Ans):- findall(Prob, (predRev(Tr,R), member(M,R), p_binom(M,P,Tr,Prob)),Res), 
(Dir = high -> split(X,Res,_,Group) ; (X1 is X + 1, split(X1, Res, Group, _))), sumlist(Group, Ans).

The sigDif fluent represents whether the rate at which a particular lab is favoured is significant under a 0.05 significance level, given the percentage of time that lab was available compared to other labs. We perform a one-sided binomial test by comparing the cumulative probabilities of X successful trials, or a more extreme result, and compare that to our significance level.

In [ ]:
% File: significantDifference.pl
holdsAt(sigDif(Requester, Source, Type, Reference, Dir, Prob, Result),T):- holdsAt(availabilityPercent(Requester, Source, Type, Reference,Percentage),T), 
holdsAt(rateFavoured(Requester, Source, Type, Reference,Successes,Trials),T), Half is Trials / 2, (Successes > Half -> Dir = high; Dir = low), 
p_cumul_binom(Successes,Percentage,Trials,Dir,Prob), ((Prob < 0.05) -> Result = "significant" ; Result = "not significant").

### History Request

At T = 1, **GG** makes a history request for patient 123 to **wellington**. As history requests are excluded from the *favours* predicate, the request event does not also cause a *favours* event to occur.

In [ ]:
?- tick(0).
?- tick(1).
?- happensAt(E,1).
?- happensAt(favours(Requester,Source, Others, Type, Reference),1).

true.
true.
E = request(2, history, 123, b'wellington', GG) .
false.

### Duplicate events

At T = 2, each of the four HIV tests that occur cause a *favours* event to also take place, as an HIV test is a valid request for the *favours* predicate, and more than one lab is available to perform an HIV test at this point. 

In [ ]:
?- tick(2).
?- happensAt(X,2).


true.
X = request(1, HIV test, 123, b'otagoBlood', AW) ;
X = request(1, HIV test, 124, b'otagoBlood', EH) ;
X = request(1, HIV test, 126, b'otagoBlood', EH) ;
X = request(1, HIV test, 127, b'wellington', EH) ;
X = withdraws(b'wellington') ;
X = favours(AW, b'otagoBlood', [b'invercargill', b'wellington'], HIV test, 1) ;
X = favours(EH, b'otagoBlood', [b'invercargill', b'wellington'], HIV test, 1) ;
X = favours(EH, b'otagoBlood', [b'invercargill', b'wellington'], HIV test, 1) ;
X = favours(EH, b'wellington', [b'invercargill', b'otagoBlood'], HIV test, 1) .

At T=3, three *favoured* predicates hold, which were initiated at the previous time period. **AW** favoured **otagoBlood** over **invercargill** and **wellington** once, while **EH** favoured **wellington** over the other two labs once, and **otagoBlood** over the other labs twice. Note that the two favours events for **EH** and **otagoBlood** are combined into one *favoured* fluent with a Count value of 2.

As shown by the rateFavoured predicate, **otagoBlood** was selected once out of one trials by **AW** for an **HIV test** with a reference value of **1**. **otagoBlood** was selected 2 times out of 3 by **EH** for an **HIV test** with a reference of 1.

In [ ]:
?- tick(3).
?- holdsAt(favoured(Requester,Source, Others, Type, Reference)=Count,3).
?- holdsAt(rateFavoured(Requester, Source, Type, Reference,Success,Trials),3).

true.
Requester = AW, Source = b'otagoBlood', Others = [ b'invercargill', b'wellington' ], Type = HIV test, Reference = 1, Count = 1 ;
Requester = EH, Source = b'otagoBlood', Others = [ b'invercargill', b'wellington' ], Type = HIV test, Reference = 1, Count = 2 ;
Requester = EH, Source = b'wellington', Others = [ b'invercargill', b'otagoBlood' ], Type = HIV test, Reference = 1, Count = 1 .
Requester = AW, Source = b'otagoBlood', Type = HIV test, Reference = 1, Success = 1, Trials = 1 ;
Requester = EH, Source = b'otagoBlood', Type = HIV test, Reference = 1, Success = 2, Trials = 3 ;
Requester = EH, Source = b'wellington', Type = HIV test, Reference = 1, Success = 1, Trials = 3 .

Considering the test request made by **AW** at time period 2, we know that all three labs were available and certified for HIV tests. Thus, considering only this time period, and only requests made by **AW**, **invercargill**, **otagoBlood** and **wellington** were all available for 33% of the total availability across all labs. Likwise, all labs were equally available during requests made by **EH**, so each have an availability percentage of 33%. 

In [ ]:
?- holdsAt(availabilityPercent(Requester, Source, Type, Reference,Percentage),3).
?- holdsAt(availabilityCount(Requester, Source, Type, Reference)=Count,3).

Requester = AW, Source = b'otagoBlood', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Requester = AW, Source = b'invercargill', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Requester = AW, Source = b'wellington', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Requester = EH, Source = b'otagoBlood', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Requester = EH, Source = b'invercargill', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Requester = EH, Source = b'wellington', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 .
Requester = AW, Source = b'otagoBlood', Type = HIV test, Reference = 1, Count = 1 ;
Requester = AW, Source = b'invercargill', Type = HIV test, Reference = 1, Count = 1 ;
Requester = AW, Source = b'wellington', Type = HIV test, Reference = 1, Count = 1 ;
Requester = EH, Source = b'otagoBlood', Type = HIV test, Reference = 1, Count = 3 ;
Requester = EH, Source

### Unequal percentages

At T = 3, **EH** requests an HIV test from **invercargill**. This creates a *favours* event, as **otagoBlood** is also available and able to perform this test.

In [ ]:
?- happensAt(X,3).

X = withdraws(b'invercargill') ;
X = request(1, HIV test, 127, b'invercargill', EH) ;
X = favours(EH, b'invercargill', [b'otagoBlood'], HIV test, 1) .

**wellington** has now been available for three requests made by **EH**, while the other labs have been available for four.

In [ ]:
?- tick(4).
?- holdsAt(availabilityCount('EH', Source, _, _)=Count,4).

true.
Source = b'wellington', Count = 3 ;
Source = b'invercargill', Count = 4 ;
Source = b'otagoBlood', Count = 4 .

While the availability percentages remain at 33% for requests made by **AW**, the percentages for requests made by **EH** have adjusted, with the values for **invercargill** and **otagoBlood** slightly higher than for **wellington**. A total of 11 lab availabilities have taken place, 4 each for **otagoBlood** and **invercargill**, and 3 for **wellington**. 3 / 11 = 0.272 and 4 / 11 = 0.364.

In [ ]:
?- holdsAt(availabilityPercent(Requester, Source, Type, Reference,Percentage),4).

Requester = AW, Source = b'otagoBlood', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Requester = AW, Source = b'invercargill', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Requester = AW, Source = b'wellington', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Requester = EH, Source = b'wellington', Type = HIV test, Reference = 1, Percentage = 0.2727272727272727 ;
Requester = EH, Source = b'invercargill', Type = HIV test, Reference = 1, Percentage = 0.36363636363636365 ;
Requester = EH, Source = b'otagoBlood', Type = HIV test, Reference = 1, Percentage = 0.36363636363636365 .

The rate at which labs are favoured has also been adjusted accordingly:

In [ ]:
?- holdsAt(rateFavoured(Requester, Source, Type, Reference,Success,Trials),4).

Requester = EH, Source = b'invercargill', Type = HIV test, Reference = 1, Success = 1, Trials = 4 ;
Requester = AW, Source = b'otagoBlood', Type = HIV test, Reference = 1, Success = 1, Trials = 1 ;
Requester = EH, Source = b'otagoBlood', Type = HIV test, Reference = 1, Success = 2, Trials = 4 ;
Requester = EH, Source = b'wellington', Type = HIV test, Reference = 1, Success = 1, Trials = 3 .

### Statistical analysis

We now wish to determine whether the number of times each lab was favoured by a specified actor is likely given the percentage of time the lab was available, if we assume a medical actor is equally likely to chose any of the suitable available labs at any given time.

Let us examine the case in which **EH** favours **otagoBlood** 2 times out of 4. According to our *availabilityPercent* fluent, **otagoBlood** was available 36.4% of the time. We wish to know whether a success rate of 2 out of 4 is likely given a probability of 36.4% of selection under each trial.

We find that the cumulative probability of 2 or less successes out of 4 is 0.86 under a binomial distribution, which is much greater than a significance level of 0.05. Therefore, the fact that **EH** selected **otagoBlood** 50% of the time out of 4 trials is not evidence to suggest that **EH** is unfairly favouring **otagoBlood** over the other labs.

In [ ]:
?- p_cumul_binom(2, 0.36363636363636, 4, low, Ans).

Ans = 0.8601188443412372 .

We can use the *sigDif* fluent to perform the one sided binomial test for us, automatically choosing which side test we wish to perform (whether we consider more extreme values to be higher or lower than the actual number of successes). The result confirms that a cumulative probability of 0.86 is not evidence to suggest **EH** is unfairly favouring **otagoBlood** over other labs. 

In [ ]:
?- holdsAt(sigDif('EH', "otagoBlood", Type, Reference, Dir, Prob, Result),4).

Type = HIV test, Reference = 1, Dir = low, Prob = 0.8601188443412334, Result = b'not significant' .

### Singular lab available

At T = 4, an HIV test request is sent to **otagoBlood** by **GG**. At this point, **otagoBlood** is the only lab available which is authorised to perform this test, so a *favours* event does not occur. **Wellington** and **invercargill** reconnect to be available from the next time period. 

In [ ]:
?- happensAt(X,4).
?- holdsAt(available(A),4), \+ holdsAt(medStaff(A)=_,4).

X = request(1, HIV test, 127, b'otagoBlood', GG) ;
X = reconnects(b'wellington') ;
X = reconnects(b'invercargill') .
A = b'otagoBlood' .

### Significant event

At T = 5, four *favours* events take place corresponding to 4 *request* events.

In [ ]:
?- tick(5).
?- happensAt(X,5).

true.
X = request(1, HIV test, 120, b'otagoBlood', GG) ;
X = request(1, HIV test, 121, b'otagoBlood', GG) ;
X = request(1, HIV test, 119, b'otagoBlood', GG) ;
X = request(1, HIV test, 118, b'otagoBlood', GG) ;
X = favours(GG, b'otagoBlood', [b'wellington', b'invercargill'], HIV test, 1) ;
X = favours(GG, b'otagoBlood', [b'wellington', b'invercargill'], HIV test, 1) ;
X = favours(GG, b'otagoBlood', [b'wellington', b'invercargill'], HIV test, 1) ;
X = favours(GG, b'otagoBlood', [b'wellington', b'invercargill'], HIV test, 1) .

They lead to a new *favoured* fluent holding, and the *rateFavoured* fluent shows that **GG** selected **otagoBlood** over the other labs four times out of four. The availability percentages for all three labs is equal at one third.

In [ ]:
?- tick(6).
?- holdsAt(favoured('GG',Source, Others, Type, Reference)=Count,6).
?- holdsAt(rateFavoured('GG', Source, Type, Reference,Success,Trials),6).
?- holdsAt(availabilityPercent('GG', Source, Type, Reference,Percentage),6).

true.
Source = b'otagoBlood', Others = [ b'wellington', b'invercargill' ], Type = HIV test, Reference = 1, Count = 4 .
Source = b'otagoBlood', Type = HIV test, Reference = 1, Success = 4, Trials = 4 .
Source = b'otagoBlood', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Source = b'wellington', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 ;
Source = b'invercargill', Type = HIV test, Reference = 1, Percentage = 0.3333333333333333 .

We find that **otagoBlood** being selected 4 times out of 4 when we expect it to be selected with a probability of 1/3 at each trial is unlikely if the probability is correct. The probability of all 4 of the trials being successes is 0.0123, which is less than our significance level of 0.05. We thus have evidence against our null hypothesis that **otagoBlood** is selected with a probability of 1/3 at each trial, and we suspect that **GG** is not choosing labs equally / fairly. 

In [ ]:
?- holdsAt(sigDif('GG', "otagoBlood", Type, Reference, Dir, Prob, Result),6).

Type = HIV test, Reference = 1, Dir = high, Prob = 0.012345679012345677, Result = b'significant' .

Of course, it may be more convenient for a doctor to send a batch of test requests to one lab at a particular lab, or one lab may be more geographically handy for sending in physical samples. In any case, we should not put too much stock in results from only a few test request occurrences. Nevertheless, we can see how a binomial test could be used to flag unlikely or unusual patterns of activity that may warrant further investigation. We would not be concerned so much with one lab being favoured overall amongst doctors, but one doctor heavily favouring a lab that is not widely used by others.

### Statistical summaries

The favoured and availability predicates above are specialised to particular medical actors, laboratories, message references and test request types. As well as being able to view statistics for specialised groups, we want to be able to calculate success rates and levels of favouritism in general, such as calculating the rate at which one lab is favoured across all doctors, and comparing this to the rate of favouritism by a specific doctor. 

The following helper functions allow us to sum counts for a given category in a list, and could be useful for calculating more general statistics.


In [ ]:
% File: groupSum.pl
key_filter(Key,Key-_).
filterSum(Filter, List, Sum):- include(key_filter(Filter), List, Filtered), maplist([_-N,N]>>true, Filtered, Nums), sumlist(Nums, Sum).
groupSum(List, SumList):- findall(Prefix, (member(M,List), M = Prefix-Num), P), sort(P,S), findall(N-Sum, (member(N, S), filterSum(N, List, Sum)), SumList).


In [ ]:
?- L = [a-1, a-4, b-2, b-9, c-22, d-11, c-16], filterSum(a, L, S).
?- L = [a-1, a-4, b-2, b-9, c-22, d-11, c-16], groupSum(L, S).

L = [ a-1, a-4, b-2, b-9, c-22, d-11, c-16 ], S = 5 .
L = [ a-1, a-4, b-2, b-9, c-22, d-11, c-16 ], S = [ a-5, b-11, c-38, d-11 ] .

It is possible to declare the behaviour of a keyword **all** in specific scenarios which creates more generalised statistics. In the example below, we calculate availability counts based on the medical actor requester, test type and message reference, but not specialised to a particular lab. While this example is functional, the behaviour of **all** cannot currently be easily applied to any other given fluent.

In [ ]:
holdsAt(availabilityCountSummary(Requester2, all, Type2, Reference2, Count2),T):-findall(Requester-Type-Reference-Count, 
holdsAt(availabilityCount(Requester, _, Type, Reference)=Count,T), Test), findall(Prefix, (member(M,Test), M = Prefix-Num), P), sort(P,S), 
findall(N-Sum, (member(N, S), filterSum(N, Test, Sum)), SumList), member(O,SumList), O = Requester2-Type2-Reference2-Count2.

We firstly view the individualised availability counts, and then with the source lab information removed:

In [ ]:
?- holdsAt(availabilityCount(Req, Source, Type, Ref)=Count, 3).
?- holdsAt(availabilityCountSummary(Req, _, Type, Ref,Count), 3).

Req = AW, Source = b'otagoBlood', Type = HIV test, Ref = 1, Count = 1 ;
Req = AW, Source = b'invercargill', Type = HIV test, Ref = 1, Count = 1 ;
Req = AW, Source = b'wellington', Type = HIV test, Ref = 1, Count = 1 ;
Req = EH, Source = b'otagoBlood', Type = HIV test, Ref = 1, Count = 3 ;
Req = EH, Source = b'invercargill', Type = HIV test, Ref = 1, Count = 3 ;
Req = EH, Source = b'wellington', Type = HIV test, Ref = 1, Count = 3 .
Req = AW, Type = HIV test, Ref = 1, Count = 3 ;
Req = EH, Type = HIV test, Ref = 1, Count = 9 .

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1527cc64-36a2-4b35-bd8b-8d493ca554fa' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>